# Logging & Tracing

## 1. Introduction

A good sign of a production-ready library or tool is how well it handles operational aspects, such as:
- how good the errors are (error handling, error messages)
- how good the logging is
- does it have any tracing capabilities

While we've already seen some of the information generated by `dlt` when executing our pipelines (the `LoadInfo` object), we haven't yet discussed how to configure production-grade logging or tracing. In this notebook, we will cover these topics.

## 2. Logging

Logging is a critical aspect of any software system. It provides a way to track the execution of the system, and it's essential for debugging and monitoring.

`dlt` has a few configurations and ways of working related to logging. Let's look at them one by one.

### 2.1. Logging Configuration

To configure logging, set the `runtime.log_level` config in `config.toml`. Typically, setting this to `INFO` is a good starting point. Let's do that now (uncomment line 3 in `config.toml`) and see how it affects the output of a pipeline:

<div class="alert alert-block alert-info">
💡 We've removed load info from the output so that only the logging output is displayed.
</div>

In [ ]:
import dlt


people = [
    {"id": "1", "name": "Warren Buffet", "country": "USA"},
    {"id": "2", "name": "Jack Ma", "country": "China"},
    {"id": "3", "name": "Rafal Brzoska", "country": "Poland"},
]

pipeline = dlt.pipeline(
    pipeline_name="dummy_source_to_duckdb",
    destination="duckdb",
    dataset_name="mydata",
)

load_info = pipeline.run(people, table_name="person")

### 2.2 Progress

Often times, it's very helpful to see how many resources (such as RAM and CPU) the pipeline is consuming.

While it's possible to track such information at the infrastructure level, depending on the setup, it might not always be possible to dig down to the pipeline level. Additioanally, it often requires setting up additional infrastructure tooling, such as Grafana.

Luckily, `dlt` provides very a simple way to track this information per-pipeline. All we have to do is specify the `progress` parameter in our pipeline definition. Let's see how it works:

In [ ]:
import dlt


people = [
    {"id": "1", "name": "Warren Buffet", "country": "USA"},
    {"id": "2", "name": "Jack Ma", "country": "China"},
    {"id": "3", "name": "Rafal Brzoska", "country": "Poland"},
]

pipeline = dlt.pipeline(
    pipeline_name="dummy_source_to_duckdb",
    destination="duckdb",
    dataset_name="mydata",
    progress="log",  # NOTE: Add this line to see resource consumption in the logs.
)

load_info = pipeline.run(people, table_name="person")

### Bonus: the near-infinite customizability of `dlt`

As in many other places, here too we can dig deeper and configure the minute details of how the progress is displayed. Besides being able to choose different display methods (eg. using Pyton's popular `tqdm` progress bar library), we can also better control the behavior of each method by using the relevant `dlt.progress` methods:

In [ ]:
import dlt
from dlt.common import logger as dlt_logger
import logging


people = [
    {"id": "1", "name": "Warren Buffet", "country": "USA"},
    {"id": "2", "name": "Jack Ma", "country": "China"},
    {"id": "3", "name": "Rafal Brzoska", "country": "Poland"},
]

pipeline = dlt.pipeline(
    pipeline_name="dummy_source_to_duckdb",
    destination="duckdb",
    dataset_name="mydata",
    # NOTE: We change "log" (which uses default parameters to dlt.progress.log())
    # to `dlt.progress.log()` with our custom parameters.
    # NOTE 2: `log_period` is the *maximum* time between progress updates. In other
    # words, if a step takes shorter, dlt will ignore this value and still emit a
    # progress update.
    progress=dlt.progress.log(log_period=5, logger=dlt_logger.LOGGER, log_level=logging.DEBUG),
)

load_info = pipeline.run(people, table_name="person")

Notice how we lost all the progress logs? That's because we've set the logging level of those messages to `DEBUG`, and by default, Jupyter only displays `INFO` and higher logs. Go back to the pipeline and set logging level to `ERROR` instead, and you will see progress logs appear again.

<div class="alert alert-block alert-info">
💡 Software engineering tip
    <br>

   Using log levels allows us to better filter out different kinds of information, thus simplifying the debugging process.
</div>

Since we're not in a production setting, let's turn off the verbose logging before proceeding.

### 2.3 Pipeline run metadata

Besides tracking the execution of the pipeline itself, `dlt` is also produces metadata about each pipeline run. This is the metadata that we've been displaying throughout the lessons so far, stored in the `LoadInfo` object returned by `pipeline.run()`.

Since this information can also be useful for tracking or debugging purposes, it's a good idea to store it somewhere. One way to do this would be to simply log this information as part of the pipeline script. However, this has the downside of having to then parse logs in order to extract pipeine run statistics, such as  duration of the `normalize` step, etc.

What we can do instead is to load this information into a separate table in the destination:

In [ ]:
import dlt


people = [
    {"id": "1", "name": "Warren Buffet", "country": "USA"},
    {"id": "2", "name": "Jack Ma", "country": "China"},
    {"id": "3", "name": "Rafal Brzoska", "country": "Poland"},
]

pipeline = dlt.pipeline(
    pipeline_name="dummy_source_to_duckdb",
    destination="duckdb",
    dataset_name="mydata",
)

load_info = pipeline.run(people, table_name="person")
pipeline.run([load_info], table_name="_load_info")

with pipeline.sql_client() as client:
    with client.execute_query("SELECT * FROM information_schema.tables") as cursor:
        data = cursor.df()

data

You can see that a few new tables have been created in the destination. Engineers can now use these tables for example to build analytics on pipeline execution*.

*Read on to learn how this could be extended with additional metadata from tracing.

## 3. Tracing

Tracing is a way to track the execution of a system across multiple components. It's especially useful in distributed systems, where a single request might be processed by multiple services.

Since this is an advanced concept, we will only briefly show how tracing information can be stored using `dlt`:

In [ ]:
pipeline.run([pipeline.last_trace], table_name="_trace")

with pipeline.sql_client() as client:
    with client.execute_query("SELECT * FROM information_schema.tables") as cursor:
        data = cursor.df()

data

## Summary

In this lesson, we've learned:

- How to configure logging in `dlt`
- How to track progress of a pipeline and its resource use
- How to work with pipeline run metadata
- How to store tracing information